# Introduction to Natural Language Processing with fastText
In this notebook we will discuss what is Natural Language Processing (NLP) and how to easily implement several projects using the library [fastText](https://github.com/facebookresearch/fastText). fastText is implemented in C++, however, there is a python wrapper, [fastText.py](https://github.com/salestock/fastText.py), that we are going to use in this notebook.

In [1]:
#Load all libraries
import os,sys  
import pandas as pd
import numpy as np
import fasttext

print(sys.version)

3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


## Text classification
The first task will be to perform text classification dataset DBPedia, which can be accessed [here](https://drive.google.com/drive/folders/0Bz8a_Dbh9Qhbfll6bVpmNUtUcFdjYmF2SEpmZUZUcVNiMUw1TWN6RDV3a0JHT3kxLVhVR2M). The dataset consists of text descriptions of 14 different classes. The training set contains 560000 reviews and the test contains 70000. 

In [ ]:
#Load train set
train_file = 'dbpedia_train.csv'
df = pd.read_csv(train_file, header=None, names=['class','name','description'])

#Load test set
test_file = 'dbpedia_test.csv'
df_test = pd.read_csv(test_file, header=None, names=['class','name','description'])

#Mapping from class number to class name
class_dict={
1:'Company',
2:'EducationalInstitution',
3:'Artist',
4:'Athlete',
5:'OfficeHolder',
6:'MeanOfTransportation',
7:'Building',
8:'NaturalPlace',
9:'Village',
10:'Animal',
11:'Plant',
12:'Album',
13:'Film',
14:'WrittenWork'
}
df['class_name'] = df['class'].map(class_dict)
df.head()

In [ ]:
#df.describe().transpose()
desc = df.groupby('class')
desc.describe().transpose()

The next step is to treat the data. As of today, the python wrapper of fastText doesn't allow dataframes or iterators as inputs to their functions (however, they are [working on it](https://github.com/salestock/fastText.py/issues/78). We have to create an intermediate file. This intermediate file doesn't have commas, non-ascii characters and everything is lowercase. The changes are based on [this script](https://github.com/facebookresearch/fastText/blob/a88344f6de234bdefd003e9e55512eceedde3ec0/classification-example.sh#L17).

In [2]:
def clean_dataset(dataframe, shuffle=False, encode=False, label_prefix='__label__'):
    # Transform train file
    df = dataframe[['name','description']].apply(lambda x: x.str.replace(',',' '))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('"',''))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('\'',' \' '))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('.',' . '))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('(',' ( '))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace(')',' ) '))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('!',' ! '))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('?',' ? '))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace(':',' '))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace(';',' '))
    df[['name','description']] = df[['name','description']].apply(lambda x: x.str.lower())
    df['class'] = label_prefix + dataframe['class'].astype(str) + ' '
    if(shuffle):
        from sklearn.utils import shuffle
        df = shuffle(df).reset_index(drop=True)
        #df.sample(frac=1).reset_index(drop=True)
    if(encode):
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii','ignore').str.decode('utf-8'))
    df['name'] = ' ' + df['name'] + ' '
    df['description'] = ' ' + df['description'] + ' '
    return df

In [ ]:
%%time
# Transform datasets
df_train_clean = clean_dataset(df, True, False)
df_test_clean = clean_dataset(df_test, False, False)

# Write files to disk
train_file_clean = 'dbpedia.train'
df_train_clean.to_csv(train_file_clean, header=None, index=False, columns=['class','name','description'] )

test_file_clean = 'dbpedia.test'
df_test_clean.to_csv(test_file_clean, header=None, index=False, columns=['class','name','description'] )

Once the dataset is cleaned, the next step is to train the classifier. 

In [ ]:
%%time
# Train a classifier
output_file = 'dp_model'
classifier = fasttext.supervised(train_file_clean, output_file, label_prefix='__label__')

Once the model is trained, we can test its accuracy. We can obtain the [percision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) of the model. High precision means that an algorithm returned substantially more relevant results than irrelevant ones, while high recall means that an algorithm returned most of the relevant results.

In [ ]:
%%time
# Evaluate classifier
result = classifier.test(test_file_clean)
print('P@1:', result.precision)
print('R@1:', result.recall)
print ('Number of examples:', result.nexamples)

The next step is to check how the model works with real sentences.

In [ ]:
sentence1 = ['Picasso was a famous painter born in Malaga, Spain. He revolutionized the art in the 20th century.']
labels1 = classifier.predict(sentence1)
class1 = int(labels1[0][0])
print("Sentence: ", sentence1[0])
print("Label: %d; label name: %s" %(class1, class_dict[class1]))

sentence2 = ['One of my favourite tennis players in the world is Rafa Nadal.']
labels2 = classifier.predict_proba(sentence2)
class2, prob2 = labels2[0][0] # it returns class2 as string
print("Sentence: ", sentence2[0])
print("Label: %s; label name: %s; certainty: %f" %(class2, class_dict[int(class2)], prob2))

sentence3 = ['Say what one more time, I dare you, I double-dare you motherfucker!']
number_responses = 3
labels3 = classifier.predict_proba(sentence3, k=number_responses)
print("Sentence: ", sentence3[0])
for l in range(number_responses):
    class3, prob3 = labels3[0][l]
    print("Label: %s; label name: %s; certainty: %f" %(class3, class_dict[int(class3)], prob3))


The model predicts sentence 1 as `Artist`, which is correct. Sentence 2 is also predicted correctly. This time we used the function `predict_proba` that retruns the certainty of the prediction as a probability. Finally, sentence 3 was not correctly classified. The correct label would be `Film`, since the sentence is from famous scene of a very good film. If by any chance, you don't know [what I'm talking about](https://www.youtube.com/watch?v=xwT60UbOZnI), well, please put your priorities in order. Stop reading this notebook, go to see Pulp Fiction, and then come back to keep learning NLP :-)

## Sentiment Analysis
Sentiment analysis is one of the most important use cases in text classification. The objective is to classify a piece of text into positive, negative and, in some cases, neutral. This is extensively used by brands to understand the perception their customers. Sentiment analysis can influence marketing campaigns, generate leads, plan product development or improeve customer service.  

In our notebook, we will use the Amazon polarity dataset, which contains 3.6 million reviews in the train set and 400.000 reviews in the test set. The reviews are positive, 1, or negative, 2. The dataset can be found [here](https://drive.google.com/drive/folders/0Bz8a_Dbh9Qhbfll6bVpmNUtUcFdjYmF2SEpmZUZUcVNiMUw1TWN6RDV3a0JHT3kxLVhVR2M). 

The first step is to prepare the dataset for the algorithm format.

In [ ]:
%%time
#Load train set
train_file = 'amazon_review_polarity_train.csv'
df_sentiment_train = pd.read_csv(train_file, header=None, names=['class','name','description'])

#Load test set
test_file = 'amazon_review_polarity_test.csv'
df_sentiment_test = pd.read_csv(test_file, header=None, names=['class','name','description'])

# Transform datasets
df_train_clean = clean_dataset(df_sentiment_train, True, False)
df_test_clean = clean_dataset(df_sentiment_test, False, False)

# Write files to disk
train_file_clean = 'amazon.train'
df_train_clean.to_csv(train_file_clean, header=None, index=False, columns=['class','name','description'] )

test_file_clean = 'amazon.test'
df_test_clean.to_csv(test_file_clean, header=None, index=False, columns=['class','name','description'] )

Once we have the data, let's train the classifier. This time instead of the default parameters we are going to use those used in the [fastText paper](https://arxiv.org/abs/1607.01759).

In [6]:
%%time
# Parameters
dim=10
lr=0.1
epoch=5
min_count=1
word_ngrams=2
bucket=10000000
thread=12
label_prefix='__label__'

# Train a classifier
output_file = 'amazon_model'
classifier = fasttext.supervised(train_file_clean, output_file, dim=dim, lr=lr, epoch=epoch,
                                 min_count=min_count, word_ngrams=word_ngrams, bucket=bucket,
                                 thread=thread, label_prefix=label_prefix)

CPU times: user 8min 28s, sys: 6.72 s, total: 8min 34s
Wall time: 1min 4s


Now let's evaluate the results

In [9]:
%%time
# Evaluate classifier
result = classifier.test(test_file_clean)
print('P@1:', result.precision)
print('R@1:', result.recall)
print ('Number of examples:', result.nexamples)

P@1: 0.9466975
R@1: 0.9466975
Number of examples: 400000
CPU times: user 4.41 s, sys: 76 ms, total: 4.48 s
Wall time: 4.47 s


Finally, let's evaluate some sentences

In [19]:
class_dict={
    1:"Positive",
    2:"Negative"
}

sentence1 = ["I hate when this situations happen"]
labels1 = classifier.predict_proba(sentence1)
class1, prob1 = labels1[0][0] # it returns class as string
print("Sentence: ", sentence1[0])
print("Label: %s; label name: %s; certainty: %f" %(class1, class_dict[int(class1)], prob1))

sentence2 = ['It is not bad, but it could be better']
labels2 = classifier.predict_proba(sentence2)
class2, prob2 = labels2[0][0] # it returns class as string
print("Sentence: ", sentence2[0])
print("Label: %s; label name: %s; certainty: %f" %(class2, class_dict[int(class2)], prob2))

Sentence:  I hate when this situations happen
Label: 2; label name: Negative; certainty: 0.916016
Sentence:  It is not bad, but it could be better
Label: 1; label name: Positive; certainty: 0.882812
